Часть 6. Модель
Вариант 2. Прогнозирование оттока пользователей (Churn Prediction)

Задача: Предсказать, какие пользователи с высокой вероятностью не вернутся на сайт.

In [ ]:
df['next_visit'] = ~df['next_visit']

In [ ]:
# содержимое данных
for col in df:
    print(col)

Для модели будем использовать фичи:

Таргет
next_visit

Количественные
pageViews
visitDuration_min
visits_before
days_last_visit


Категориальные
Source
UTMSource_category
weekday - уже закодировано
day_period
visitDuration_category
operatingSystem_category
type_device
regionCity - используем частотное кодирование, много значений


Бинарные
isNewUser
bounce
has_registration
registration_left
is_weekend


МОДЕЛЬ

In [ ]:
df_model = df.copy()

In [ ]:
df_model.drop(
    [
        'visitID',
        'date',
        'dateTime',
        'dateTimeUTC',
        'startURL',
        'endURL',
        'clientID',
        'counterUserIDHash',
        'mobilePhone',
        'operatingSystemRoot',
        'operatingSystem',
        'browser',
        'UTMCampaign',
        'UTMContent',
        'UTMMedium',
        'UTMSource',
        'UTMTerm',
        'visitDuration',
        'total_regs',
        'regionCountry',
        'referer',
        'lastTrafficSource',
        'referer_domain',
        'days_last_visit'
        ] , axis =1 ,inplace = True)

In [ ]:
df_model.info()

Кодируем категориальные признаки, масштабируем количественные

In [ ]:
# применяем fit_transform
# Определяем числовые столбцы
numeric_cols = df_model.select_dtypes(include='number').columns

# Создаём копии, чтобы не терять остальные признаки
df_model_scaled = df_model.copy()

# Масштабируем только числовые колонки
scaler = StandardScaler()
df_model_scaled[numeric_cols] = scaler.fit_transform(df_model[numeric_cols])


In [ ]:
# используем частотное кодирование

city_freq = df_model['regionCity'].value_counts(normalize=True)
df_model['city_freq'] = df_model['regionCity'].map(city_freq)
df_model.drop(['regionCity'] , axis =1 ,inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
selected_columns = ['Source',
                   'day_period',
                   'visitDuration_category',
                   'type_device',
                   'UTMSource_category',
                   'operatingSystem_category']

encoded_columns = {}
for col in selected_columns:
    df_model[col] = label_encoder.fit_transform(df_model[col])
    # Сохраняем маппинг
    encoded_columns[col] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [ ]:
df_model.info()

In [ ]:
df_model.head(10)

In [ ]:
X = df_model.drop(columns=['next_visit'])
y = df_model['next_visit']

# разобьем данные на обучающую и тестовый выборки

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x=y, palette='viridis')
plt.title('Распределение классов Revenue')
plt.show()

print("Соотношение классов:\n", df['next_visit'].value_counts(normalize=True))
print(round(len(df['next_visit'])/len(df[df['next_visit'] == 1]), 1))

In [ ]:
# метрики моделей

metrics = pd.DataFrame(index=['precision', 'recall', 'f1', 'time'])

Модель 1. Бэггинг

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import time

models = {
    "BaggingClassifier": BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=50, random_state=42),
    "ExtraTreesClassifier": ExtraTreesClassifier(n_estimators=50, random_state=42)
}

# обучаем модели и выводим classification_report
start = time.time()
for name, model in models.items():
    print("="*30)
    print(f"Model: {name}")
    model.fit(X_train, y_train)  # обучаем модель

    y_pred = model.predict(X_test)  # предсказываем классы
    stop = time.time()
    # выводим classification_report
    print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['bagging'] = [precision, recall, f1, elapsed_time]



Модель 2. AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
start = time.time()

# Создаём и обучаем AdaBoost
ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)
ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_ada))


precision = precision_score(y_test, y_pred_ada)
recall = recall_score(y_test, y_pred_ada)
f1 = f1_score(y_test, y_pred_ada)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['AdaBoost'] = [precision, recall, f1, elapsed_time]

Модель 3. GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
start = time.time()

gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train, y_train)
y_pred_gbm = gbm.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_gbm))

precision = precision_score(y_test, y_pred_gbm)
recall = recall_score(y_test, y_pred_gbm)
f1 = f1_score(y_test, y_pred_gbm)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['GBM'] = [precision, recall, f1, elapsed_time]

Модель 4. XGBoost

In [ ]:
import xgboost as xgb
start = time.time()

xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, eval_metric="logloss")
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_xgb))

precision = precision_score(y_test, y_pred_xgb)
recall = recall_score(y_test, y_pred_xgb)
f1 = f1_score(y_test, y_pred_xgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['XGBoost'] = [precision, recall, f1, elapsed_time]


Модель 5. LightGBM

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,   # Без ограничения глубины
    'num_leaves': 31,  # Стандартное значение
    'min_data_in_leaf': 5,  # Сделаем меньше, чтобы больше разбиений прошло
    'min_gain_to_split': 0.0,  # Уберем ограничение на минимальное улучшение сплита
    'feature_fraction': 0.8,
    'verbose':0
}
start = time.time()

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)
y_pred_lgb = model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM'] = [precision, recall, f1, elapsed_time]

Модель 6. CatBoost

In [ ]:
df_cat = df.copy()
df_cat.drop(
    [
        'visitID',
        'date',
        'dateTime',
        'dateTimeUTC',
        'startURL',
        'endURL',
        'clientID',
        'counterUserIDHash',
        'mobilePhone',
        'operatingSystemRoot',
        'operatingSystem',
        'browser',
        'UTMCampaign',
        'UTMContent',
        'UTMMedium',
        'UTMSource',
        'UTMTerm',
        'visitDuration',
        'total_regs',
        'regionCountry',
        'referer',
        'lastTrafficSource',
        'referer_domain',
        'days_last_visit'
        ] , axis =1 ,inplace = True)

In [ ]:
X_cat = df_cat.drop(columns=['next_visit'])
y_cat = df_cat['next_visit']
# разделяем данные на тренировочную (80%) и тестовую (20%) выборки с учетом дисбаланса (stratify=y)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, stratify=y, random_state=42)

import catboost as cb
# определяем категориальные признаки
cat_features = [
    'Source',
    'day_period',
    'visitDuration_category',
    'type_device',
    'UTMSource_category',
    'operatingSystem_category',
    'regionCity'
    ]
start = time.time()

# создаём и обучаем CatBoostClassifier
cat_model = cb.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, random_state=42, verbose=0)
cat_model.fit(X_train_cat, y_train_cat, cat_features=cat_features)
y_pred_cat = cat_model.predict(X_test_cat)

stop = time.time()
print(classification_report(y_test_cat, y_pred_cat))

precision = precision_score(y_test_cat, y_pred_cat)
recall = recall_score(y_test_cat, y_pred_cat)
f1 = f1_score(y_test_cat, y_pred_cat)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['CatBoost'] = [precision, recall, f1, elapsed_time]


In [ ]:
metrics

Вывод:

лучше всего справилась LightGBM

Precision = 0.87
— Очень высокий показатель: модель почти не делает ложных положительных предсказаний, если она говорит "пользователь вернётся" — скорее всего, так и есть.

Recall = 0.65
— Средний показатель: модель находит не всех, кто реально вернулся, пропуская 36% вернувшихся.

F1 = 0.74
— В целом модель работает хорошо, но есть дисбаланс: точность высокая, полнота — нет.

Time = 2.44 секунды
— Нормальное время

Модель осторожная: делает предсказания только в "уверенных" случаях.
Хороша для сценариев, где важно не ошибиться с возвратом (например, направить дорогую рекламу только тем, кто точно вернётся).
Но если нужно найти всех потенциально вернувшихся — recall надо повысить

Попробуем теперь избавиться от неважных фичей и попробовать ту же модель

Модель 7. LightGBM без неважных фичей

In [ ]:
df_lgb = df_model.copy()
df_lgb.drop([
    'bounce',
    'registration_left',
    'has_registration',
    'weekday',
    'is_weekend',
    'visits_before'
], axis =1 ,inplace = True)

In [ ]:
df_lgb.info()

In [ ]:
# разобьем данные на обучающую и тестовый выборки

X = df_lgb.drop(columns=['next_visit'])
y = df_lgb['next_visit']

# разобьем данные на обучающую и тестовый выборки

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

In [ ]:

params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,   # Без ограничения глубины
    'num_leaves': 31,  # Стандартное значение
    'min_data_in_leaf': 5,  # Сделаем меньше, чтобы больше разбиений прошло
    'min_gain_to_split': 0.0,  # Уберем ограничение на минимальное улучшение сплита
    'feature_fraction': 0.8,
    'verbose':0
}
start = time.time()

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)
y_pred_lgb = model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM_imp_feature'] = [precision, recall, f1, elapsed_time]

Вывод:
без неважных признаков модель стала совсем немного лучше по recall, но значимо хуже по precision

Попробуем отдать LightGBM самостоятельно кодировать признаки

Модель 8. LightGBM с самостоятельной обработкой категориальных признаков

In [ ]:
cat_features = [
    'Source',
    'day_period',
    'visitDuration_category',
    'type_device',
    'UTMSource_category',
    'operatingSystem_category',
    'regionCity'
    ]  # категориальные фичи

df_lgb_categ = df.copy()

df_lgb_categ.drop(
    [
        'visitID',
        'date',
        'dateTime',
        'dateTimeUTC',
        'startURL',
        'endURL',
        'clientID',
        'counterUserIDHash',
        'mobilePhone',
        'operatingSystemRoot',
        'operatingSystem',
        'browser',
        'UTMCampaign',
        'UTMContent',
        'UTMMedium',
        'UTMSource',
        'UTMTerm',
        'visitDuration',
        'total_regs',
        'regionCountry',
        'referer',
        'lastTrafficSource',
        'referer_domain',
        'days_last_visit'
        ] , axis =1 ,inplace = True)

X = df_lgb_categ.drop(columns=['next_visit'])
y = df_lgb_categ['next_visit']

# разобьем данные на обучающую и тестовый выборки

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

# Преобразуем их в тип category
for col in cat_features:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')


In [ ]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,   # Без ограничения глубины
    'num_leaves': 31,  # Стандартное значение
    'min_data_in_leaf': 5,  # Сделаем меньше, чтобы больше разбиений прошло
    'min_gain_to_split': 0.0,  # Уберем ограничение на минимальное улучшение сплита
    'feature_fraction': 0.8,
    'verbose':0
}
start = time.time()

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)
y_pred_lgb = model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM_categ'] = [precision, recall, f1, elapsed_time]

In [ ]:
metrics

Вывод:
Качество не поменялось

Попробуем подбор гиперпараметров

Модель 9. LightGBM с подбором гиперпараметров

In [ ]:
X = df_model.drop(columns=['next_visit'])
y = df_model['next_visit']

# разобьем данные на обучающую и тестовый выборки

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

# Создаём модель
model = lgb.LGBMClassifier(objective='binary', metric='auc', verbose=0)

# Гиперпараметры для перебора
param_dist = {
    'num_leaves': [15, 31, 63, 127],
    'max_depth': [-1, 5, 10, 20, 50],
    'learning_rate': [0.005, 0.01, 0.05, 0.1],
    'min_child_samples': [5, 10, 20, 50],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1]
}

# Используем F1 как метрику для оптимизации
scorer = make_scorer(f1_score)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring=scorer,
    n_iter=50,           # Кол-во вариантов (можно больше, если есть время)
    cv=3,                # Кросс-валидация
    verbose=1,
    n_jobs=-1,
    random_state=42
)
start = time.time()
random_search.fit(X_train, y_train)

# Предсказания и результат
best_model = random_search.best_estimator_
y_pred_lgb = best_model.predict(X_test)
stop = time.time()
print("Best parameters:", random_search.best_params_)
print("F1 score on test set:", f1_score(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM_giperparam'] = [precision, recall, f1, elapsed_time]

In [ ]:
metrics

Вывод:
почти не улучшилось :)

Попробуем удалить выбросы и нули

Модель 10. LightGBM для увеличения recall

In [ ]:
X = df_model.drop(columns=['next_visit'])
y = df_model['next_visit']

# разобьем данные на обучающую и тестовый выборки

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,   # Без ограничения глубины
    'num_leaves': 31,  # Стандартное значение
    'min_data_in_leaf': 5,  # Сделаем меньше, чтобы больше разбиений прошло
    'min_gain_to_split': 0.0,  # Уберем ограничение на минимальное улучшение сплита
    'feature_fraction': 0.8,
    'verbose':0,
    'scale_pos_weight': 2.3
}
start = time.time()

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)
y_pred_lgb = model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM_scale'] = [precision, recall, f1, elapsed_time]

In [ ]:
metrics

Модель 11. LightGBM без выбросов

In [ ]:
df_model = df_model.loc[
    (df_model['visitDuration_min'] != 0) &
    (df_model['pageViews']        != 0)
]

In [ ]:
X = df_model.drop(columns=['next_visit'])
y = df_model['next_visit']

# разобьем данные на обучающую и тестовый выборки

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,   # Без ограничения глубины
    'num_leaves': 31,  # Стандартное значение
    'min_data_in_leaf': 5,  # Сделаем меньше, чтобы больше разбиений прошло
    'min_gain_to_split': 0.0,  # Уберем ограничение на минимальное улучшение сплита
    'feature_fraction': 0.8,
    'verbose':0,
    'scale_pos_weight': 2.3
}
start = time.time()

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)
y_pred_lgb = model.predict(X_test)

stop = time.time()
print(classification_report(y_test, y_pred_lgb))


precision = precision_score(y_test, y_pred_lgb)
recall = recall_score(y_test, y_pred_lgb)
f1 = f1_score(y_test, y_pred_lgb)
elapsed_time = round((stop - start), 4)

print(f'время работы алгоритма: {elapsed_time:.1f} секунд')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

metrics['LightGBM_no_zeros'] = [precision, recall, f1, elapsed_time]

In [ ]:
metrics

In [ ]:

def find_best_threshold_df(y_true, y_pred_proba, plot=True):
    thresholds = np.linspace(0, 1, 100)
    rows = []
    for thr in thresholds:
        y_pred = (y_pred_proba >= thr).astype(int)
        rows.append({
            'threshold': thr,
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall':    recall_score(y_true, y_pred, zero_division=0),
            'f1':        f1_score(y_true, y_pred, zero_division=0)
        })
    df_scores = pd.DataFrame(rows)
    best = df_scores.loc[df_scores['f1'].idxmax()]

    if plot:
        plt.figure(figsize=(8,4))
        plt.plot(df_scores['threshold'], df_scores['precision'], label='Precision')
        plt.plot(df_scores['threshold'], df_scores['recall'],    label='Recall')
        plt.plot(df_scores['threshold'], df_scores['f1'],        label='F1')
        plt.axvline(best['threshold'], color='gray', linestyle='--', label=f"Best thr={best['threshold']:.2f}")
        plt.xlabel('Threshold'); plt.ylabel('Score')
        plt.legend(); plt.grid(True); plt.tight_layout()
        plt.show()

    return best['threshold'], df_scores

# ---------------------------------------
# Как использовать:
y_pred_proba = model.predict_proba(X_test)[:, 1]

best_thr, scores_df = find_best_threshold_df(y_test, y_pred_proba)

# Выводим лучший порог и его метрики:
print("Лучший порог:", best_thr)
print(scores_df.loc[scores_df['threshold'] == best_thr])

# Или вывести весь DataFrame:
print(scores_df.head())    # первые 5 строк
print(scores_df.tail())    # последние 5 строк
